In [ ]:
import os
import re
import glob

In [ ]:
import pandas as pd
from sympy import sympify

In [ ]:
from pydantic import BaseModel
from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder

In [ ]:
from insolver import InsolverDataFrame
from insolver.transforms import InsolverTransform, load_transforms
from insolver.wrappers import InsolverGLMWrapper, InsolverGBMWrapper
from insolver.serving import utils
from insolver.configs.settings import FORMULA, FORMULA_CALCULATION, N_CORES, VARIABLES_LIST

For logging

In [ ]:
import logging
from logging.handlers import RotatingFileHandler
from time import time
from multiprocessing import Pool

add new features

In [ ]:
if os.environ['models_folder'] is not None:
    models_folder = os.environ['models_folder']
if os.environ['transforms_folder'] is not None:
    transforms_folder = os.environ['transforms_folder']
if os.environ['config_file'] is not None:
    config_file = os.environ['config_file']

Logging

In [ ]:
handler = RotatingFileHandler('app.log', maxBytes=100000, backupCount=5)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

print(FORMULA_CALCULATION)<br>
print(FORMULA)<br>
print(VARIABLES_LIST)<br>
print(N_CORES)

In [ ]:
path_models = models_folder
path_transforms = transforms_folder

In [ ]:
models = []
transforms = []

var 1 to upload models

In [ ]:
models = [f for f in glob.glob(path_models + '/*')]
models.sort()

var 2 to upload models

In [ ]:
transforms = [f for f in glob.glob(path_transforms + '/*')]
transforms.sort()

print('models:', models)<br>
print('transforms:', transforms)

In [ ]:
dict_variables = {}
if FORMULA_CALCULATION:
    dict_variables = {i: 1 for i in VARIABLES_LIST}

list of key objects  of models

In [ ]:
mlist = []
tlist = []
vlist = []
itlist = []

Load models once

In [ ]:
for i, model_path in enumerate(models):
    # Load model
    # print(i, model_path, models[i], transforms[i])
    model = utils.load_pickle_model(models[i])
    if model and model.algo == 'gbm':
        model = InsolverGBMWrapper(backend=model.backend, load_path=models[i])
    elif model and model.algo == 'glm':
        model = InsolverGLMWrapper(backend='sklearn', load_path=models[i])
    else:
        model = InsolverGLMWrapper(backend='h2o', load_path=models[i])
    mlist.append(model)

    # load transformations
    transformations = load_transforms(transforms[i])
    tlist.append(transformations)
    regex = re.split(r'[\s*+()/_\s]\s*', models[i])
    current_variable_model = list(filter(None, regex))[-2]
    regex = re.split(r'[\s*+()/_\s]\s*', transforms[i])
    current_variable_transform = list(filter(None, regex))[-2]
    vlist.append(current_variable_model)
    # print(vlist)

In [ ]:
def pool_inference(pack):
    i = pack[1]
    df = pack[0]
    # print('index', i)
    insdataframe = InsolverDataFrame(df)
    instransforms = InsolverTransform(insdataframe, tlist[i])
    instransforms.ins_transform()
    predicted = mlist[i].predict(instransforms)
    return [i, predicted[0]]

In [ ]:
app = FastAPI()

In [ ]:
class Data(BaseModel):
    df: dict

In [ ]:
@app.get("/")
def index():
    return "API for predict service"